In [ ]:
#works with cv virtual environment

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, load_img

import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os import system
import os
import random

import imageio

import sys
import glob
import cv2

Using Theano backend.
WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named mkl


In [2]:
folder_path = '/home/niaki/Code/hpatches/hpatches-benchmark/data/hpatches-release'
patch_size = 65

In [3]:
# all types of patches 
tps = ['ref','e1','e2','e3','e4','e5','h1','h2','h3','h4','h5',\
       't1','t2','t3','t4','t5']

class hpatches_sequence:
    """Class for loading an HPatches sequence from a sequence folder"""
    itr = tps
    def __init__(self,base):
        name = base.split('/')
        self.name = name[-1]
        self.base = base
        for t in self.itr:
            im_path = os.path.join(base, t+'.png')
            im = cv2.imread(im_path,0)
            self.N = im.shape[0]/patch_size
            setattr(self, t, np.split(im, self.N))

In [4]:
seqs = glob.glob(folder_path+'/*')
seqs = [os.path.abspath(p) for p in seqs]

In [5]:
len(seqs)

116

In [6]:
descr_name = 'vae_0.0.0.9'
base_dir = '/home/niaki/Code/sundry_segments/weights'
model_version = '0.0.0.9_lr0.0001_50moreepochs'
vae_patch_size = 56

In [7]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [8]:
latent_dim = 128

encoder_inputs = keras.Input(shape=(vae_patch_size, vae_patch_size, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 56, 56, 1)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 14, 14, 64)   18496       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 7, 7, 64)     36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
flatten (F

In [9]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3136)              404544    
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 64)        36928     
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 56, 56, 32)        18464     
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 56, 56, 1)         289       
Total para

In [10]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = encoder(data)
            reconstruction = decoder(z)
            reconstruction_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data, reconstruction)
            )
            reconstruction_loss *= 28 * 28
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

In [11]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

In [12]:
vae.load_weights(base_dir + '/vae_' + model_version + '.h5')

In [13]:
print(encoder.output_shape)

[(None, 128), (None, 128), (None, 128)]


In [ ]:
# for seq_path in seqs:
#     seq = hpatches_sequence(seq_path)
#     path = os.path.join(descr_name,seq.name)
#     if not os.path.exists(path):
#         os.makedirs(path)
#     descr = np.zeros((seq.N,2)) # trivial (mi,sigma) descriptor
#     for tp in tps:
#         print(seq.name+'/'+tp)
#         for i,patch in enumerate(getattr(seq, tp)):
#             mi = np.mean(patch) # trivial (mi,sigma) descriptor
#             sigma = np.std(patch) # trivial (mi,sigma) descriptor
#             descr[i] = np.array([mi,sigma])
#         np.savetxt(os.path.join(path,tp+'.csv'), descr, delimiter=',') # X is an array

In [14]:
for seq_path in seqs:
    seq = hpatches_sequence(seq_path)
    path = os.path.join("hpatches_extracted_descrs", descr_name, seq.name)
    if not os.path.exists(path):
        os.makedirs(path)

    descr_size_per_patch = 128
    descr = np.zeros((seq.N,descr_size_per_patch)) # trivial (mi,sigma) descriptor   
    print(seq.name)
    for tp in tps:
#         print(seq.name+'/'+tp)
        for i,patch in enumerate(getattr(seq, tp)):
            patch = patch / 255.0
#             print(patch)
            patch_crop = patch[4: 60, 4: 60]
            patch_crop_encoded = encoder.predict(np.expand_dims(np.expand_dims(patch_crop, axis=-1), axis=0))
            patch_crop_encoded_flat = patch_crop_encoded[2].flatten()
#             print(patch_crop.shape)
#             print(patch_crop_encoded[2].shape)
#             print(patch_crop_encoded_flat.shape)
#             print()
            descr[i] = patch_crop_encoded_flat
        np.savetxt(os.path.join(path,tp+'.csv'), descr, delimiter=',') # X is an array

i_kurhaus
i_veggies
i_fruits
v_grace
v_bark
i_boutique
v_tempera
i_dome
v_busstop
v_vitro
v_bip
v_strand
i_salon
i_brooklyn
i_steps
v_samples
v_dirtywall
i_bologna
i_fenis
v_wormhole
v_posters
i_londonbridge
i_indiana
i_objects
i_nijmegen
v_cartooncity
i_fog
v_yard
i_tools
i_nescafe
i_gonnenberg
v_gardens
i_lionnight
v_home
i_autannes
i_whitebuilding
v_wall
i_smurf
v_yuri
i_kions
i_greentea
i_pinard
v_bricks
v_charing
v_artisans
v_there
v_wounded
v_soldiers
v_bees
i_zion
i_porta
v_fest
v_calder
v_machines
i_yellowtent
v_sunseason
i_resort
i_leuven
i_crownday
i_village
v_dogman
i_ktirio
v_blueprint
i_ajuntament
i_bridger
i_greenhouse
v_london
i_books
i_chestnuts
v_courses
v_boat
v_coffeehouse
i_crownnight
i_toy
v_astronautis
i_dc
i_nuts
v_abstract
i_ski
i_santuario
v_beyus
v_tabletop
v_underground
v_feast
i_lionday
v_weapons
v_bird
v_maskedman
i_table
v_adam
i_parking
v_laptop
v_churchill
i_pool
i_castle
i_melon
i_contruction
v_wapping
v_talent
i_miniature
v_man
v_apprentices
v_eastsout